In [1]:
!wget -c https://github.com/euphoris/datasets/raw/master/neurips.zip

--2021-04-20 10:50:12--  https://github.com/euphoris/datasets/raw/master/neurips.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/euphoris/datasets/master/neurips.zip [following]
--2021-04-20 10:50:13--  https://raw.githubusercontent.com/euphoris/datasets/master/neurips.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1357662 (1.3M) [application/zip]
Saving to: ‘neurips.zip’

neurips.zip         100%[===================>]   1.29M  --.-KB/s    in 0.07s   

2021-04-20 10:50:13 (17.9 MB/s) - ‘neurips.zip’ saved [1357662/1357662]



In [2]:
import pandas as pd 
df = pd.read_csv('neurips.zip')

In [3]:
df.head()

,year,title,abstract
0,2007,Competition Adds Complexity,It is known that determinining whether a DEC-P...
1,2007,Efficient Principled Learning of Thin Junction...,We present the first truly polynomial algorith...
2,2007,Regularized Boost for Semi-Supervised Learning,Semi-supervised inductive learning concerns ho...
3,2007,Simplified Rules and Theoretical Analysis for ...,We show that under suitable assumptions (prima...
4,2007,Predicting human gaze using low-level saliency...,"Under natural viewing conditions, human observ..."


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [5]:
cv = TfidfVectorizer(stop_words='english', max_features=2000)
x = cv.fit_transform(df.abstract)

In [6]:
words = cv.get_feature_names()

In [7]:
# 젠심 라이브러리 설치 
!pip install gensim

In [8]:
from gensim.matutils import Sparse2Corpus

In [10]:
# 젠심에서는 자기들이 사용하는 corpus 형태로 문서 단어행렬을 변환할수 있게 함수제공 
# Sparse2Corpus 에 넣을때는 기존 tdm 을 transpose로 넣어주어야 함 
corpus = Sparse2Corpus(x.T)

In [11]:
corpus[0]

[(1209, 0.19719051171057086),
 (1250, 0.1878641742740461),
 (265, 0.11655716303382495),
 (495, 0.21779310927686704),
 (316, 0.16096846113531893),
 (321, 0.25275451313424946),
 (317, 0.3778227141887415),
 (1573, 0.35429796717697476),
 (650, 0.32556474378711336),
 (1346, 0.3395230603787923),
 (1759, 0.32398854042493375),
 (767, 0.18302469761238765),
 (1756, 0.1205808084872785),
 (1219, 0.2021297737105109),
 (1282, 0.1848866146903614),
 (984, 0.2396176930939823)]

In [12]:
# 단어를 사전형태로 만들어둘 필요가 있다 
id2token = dict(enumerate(words))

In [14]:
id2token[9]

'abstract'

In [15]:
# 젠심 자체적으로 전환하는 변환 방법 
# 일단 토크나이저를 먼저 만들어야 함 
import re 
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
# 정규표현식으로 특정 단어를 추출하고 영어 스탑워드를 할것임 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [16]:
token_re = re.compile(r'\b\w{2,}\b', re.UNICODE) # \b 단어경계 \w문자 {2,}2글자이상 있는 경우 
# re.UNICODE(영어 뿐만 아니라 한국어도 적용되게 하는 코드)

# 토큰나이저 함수 만들기 
def tokenizer(text):
  text = text.lower()
  words=[]
  for word in token_re.findall(text):
    if word not in ENGLISH_STOP_WORDS:
      words.append(word)
  return words

In [18]:
# 토크나이저 함수 기존 텍스트 데이터를 함수 적용해서 리스트에 다 담아주기 
docs = []
for text in df.abstract:
  doc = tokenizer(text)
  docs.append(doc)

In [19]:
# 토큰화가 다 끝났고 단어 사전을 만들어 주는 작업 
from gensim.corpora.dictionary import Dictionary

In [20]:
dic = Dictionary(docs)

In [22]:
# 이 딕셔너리에서는 일정한 기준으로 단어들을 잘라낼수 있다. 
# filter_extremes 의 파라미터 
# no_below 너무 일부 문서에만 나온 단어들 제거 
# no_above 너무 흔하게 나오는 단어들 제거 
# 즉 최대 10개 문서에서는 나와야 되고, 90%의 문서에서 나온 단어면 제외 
dic.filter_extremes(no_below=10, no_above=0.9)

In [23]:
dic[100]

'existing'

In [28]:
docs[0]

['known',
 'determinining',
 'dec',
 'pomdp',
 'cooperative',
 'partially',
 'observable',
 'stochastic',
 'game',
 'posg',
 'cooperative',
 'strategy',
 'positive',
 'expected',
 'reward',
 'complete',
 'nexp',
 'known',
 'cooperation',
 'affected',
 'complexity',
 'competitive',
 'posgs',
 'complexity',
 'determining',
 'team',
 'positive',
 'expected',
 'reward',
 'strategy',
 'complete',
 'class',
 'nexp',
 'oracle',
 'np']

In [29]:
# 토큰들만들어 있는 docs를 gensim 에서 사용하는 corpus 형식으로 바꿔야함 
# 앞서 만들어 놓았던 dic 단어사전에 doc2bow라는 함수를 사용 
# 단어들을 몇개씩 나왔는지 합쳐서 세서 묶어놓음 순서는 무시 
corpus = []
for doc in docs:
  bow = dic.doc2bow(doc)
  corpus.append(bow)

In [31]:
# sklearn의 CountVectorizer 와 유사한 효과 
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 2),
 (5, 2),
 (6, 1),
 (7, 2),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 2),
 (16, 2),
 (17, 1),
 (18, 2)]

In [32]:
# LDA 모델에 넣어서 주제분석 시행 
from gensim.models.ldamodel import LdaModel

In [33]:
# 데이터 분할 
from sklearn.model_selection import train_test_split
train_corpus, valid_corpus = train_test_split(corpus, test_size=0.1, random_state=5432)

In [34]:
# 경고창 무시 
import warnings 
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [35]:
# train_corpus 넣어주고, 번호와 단어를 짝지어준것을 넣어줘야함 우리는 만들어놓은 dic 사전을 넣으면 된다 
# 주제의 수도 정해 주어야 한다. 
model = LdaModel(corpus=train_corpus, id2word=dic, num_topics=100, random_state=1234)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [36]:
# 모델의 성능 측정 
loss = model.log_perplexity(valid_corpus)
loss

# 원래 - 를 붙이면 crossentropy 교차 엔트로피가 된다 
# 보통 교차 엔트로피를 감소 시킨다. 
# 하지만 여기는 - 가 붙어 있으니까 증가를 시켜야함 
# 보통 0에 가까운 쪽이 좋은 쪽 

-20.292589544631266

In [37]:
# 모델의 성능을 높히기 위해 while 문을 써서 모델을 업데이트 
import numpy 
old_loss = -numpy.inf
while loss > old_loss + 0.1:
  model.update(train_corpus)
  old_loss = loss
  loss = model.log_perplexity(valid_corpus)
  print(loss)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


-18.418338372680363
-17.330340104100443
-16.732485037604523
-16.391526544413537
-16.177193612285368
-16.033237642340254
-15.929959250736918
-15.853177828752493


In [ ]:
# log_perplexity 이것은 로그 라이클리후드 라는 지표 (혼란도 라고도 한다.)

In [38]:
# model 저장 하기 
model.save('lda-model')

In [39]:
# 압축 
# lda-model로 시작하는애들 다 압축해뿌리라 
!zip mylad.zip lda-model*

  adding: lda-model (deflated 81%)
  adding: lda-model.expElogbeta.npy (deflated 72%)
  adding: lda-model.id2word (deflated 47%)
  adding: lda-model.state (deflated 47%)


In [40]:
# lda-model loading
model = LdaModel.load('lda-model')

In [42]:
# 앞서 분석한 LDA 모델의 결과 보기 
# 0번 토픽에 자주나오는 단어를 보여줌 
model.show_topic(0) 

[('algorithm', 0.032322522),
 ('problem', 0.022216178),
 ('optimal', 0.019816509),
 ('algorithms', 0.016037669),
 ('convex', 0.013535194),
 ('statistical', 0.012864659),
 ('optimization', 0.011256773),
 ('number', 0.011023235),
 ('performance', 0.009503619),
 ('experiments', 0.009414744)]

In [43]:
# 주제분석과 관련된 토픽 찾기 
# 앞서 만들어 놓았던 사전에서 token2id 활용 
# 단어를 넣으면 번호를 알수 있음 
dic.token2id['topic']

307

In [45]:
# 307번 단어가 어느 토픽에서 많이 나오는지 찾기 
# 307번에 관련된 토픽을 찾는 것 , 307번 단어가 1% 이상 나온 토픽을 찾아줘 
model.get_term_topics(307, 0.01)
# 8번 토픽에서 307번 단어가 7% 나옴 

[(8, 0.070252545)]

In [47]:
model.show_topic(8)

[('topic', 0.07027377),
 ('model', 0.044330813),
 ('dirichlet', 0.031132543),
 ('latent', 0.030145569),
 ('topics', 0.02780345),
 ('models', 0.026164815),
 ('lda', 0.024907071),
 ('document', 0.024669547),
 ('word', 0.02426642),
 ('words', 0.023324741)]

In [48]:
new_text = '''We describe latent Dirichlet allocation (LDA), a generative probabilistic model for collections of
discrete data such as text corpora. LDA is a three-level hierarchical Bayesian model, in which each
item of a collection is modeled as a finite mixture over an underlying set of topics. Each topic is, in
turn, modeled as an infinite mixture over an underlying set of topic probabilities. In the context of
text modeling, the topic probabilities provide an explicit representation of a document. We present
efficient approximate inference techniques based on variational methods and an EM algorithm for
empirical Bayes parameter estimation. We report results in document modeling, text classification,
and collaborative filtering, comparing to a mixture of unigrams model and the probabilistic LSI
model.'''

In [51]:
# 새로운 문서의 주제 찾기 
doc = tokenizer(new_text)
bow = dic.doc2bow(doc)

In [52]:
model.get_document_topics(bow)

[(1, 0.094580114),
 (8, 0.44311956),
 (12, 0.068104),
 (14, 0.08350934),
 (22, 0.065102346),
 (44, 0.017677667),
 (54, 0.062232662),
 (78, 0.0505859),
 (96, 0.10227152)]

In [ ]:
# LDAvis를 통한 결과 시각화 

In [ ]:
# LDAvis 설치 
# 코렙같은 경우에는 구버전으로 까는것을 추천 
# !pip install pyLDAvis==2.1.2

In [54]:
import pyLDAvis.gensim

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [55]:
# 우리는 주피터 노트북을 사용하니 세팅 
pyLDAvis.enable_notebook()

In [57]:
# model , corpus(만든 단어목록), dic(사전), sort_topics는 False 로 맞춰줌
p = pyLDAvis.gensim.prepare(model, corpus, dic, sort_topics=False)

In [58]:
pyLDAvis.display(p)

In [60]:
# 응집도의 계산 (높으면 높을수록 좋다)
from gensim.models import CoherenceModel

In [62]:
# model 넣어주고, corpus는 전체를 넣어줌, token화한 텍스트를 넣어줌, 만든 사전 넣어주고 응집도 지표 c_v를 넣어줌 
coh = CoherenceModel(model=model, corpus=corpus, texts=docs, dictionary=dic, coherence='c_v')

In [63]:
coh.get_coherence()

0.3535195687398563

In [ ]:
# 응집도가 놓으면 높을수록 좋은 지표 
# 주제분석을 할때, 주제의 수를 100개로 했는데 50개로 바꿔서 돌려봐서 응집도를 비교 
# 50개를 했는데 응집도가 높다면 그것을 기준으로 topic을 50개로 줄일수 있다. 
# 학습을 시킬때 응집도가 높아지는 방안으로 학습을 반복하게 하든가 가 있음 

In [ ]:
# 다양도 (주제별로 얼마나 다양한가를 보는 것)

In [64]:
topn = 25
top_words = set()
# set 중복되는 원소는 들어가지 않는다 중복을 제외하고 순수하게 단어가 몇개 사용 됬는지 알수 있다. 

for topic in range(model.num_topics):
  for word, prob in model.show_topic(topic, topn=topn):
    top_words.add(word)
    # 없는 단어는 추가 있는 단어는 아무것도 하지않는다. 

In [66]:
len(top_words)

1072

In [67]:
1072 / 2500

0.4288

In [ ]:
# 주제의 다양도는 42% 다양도도 높을수록 좋은 것 
# 학습과정에서 다양도가 충분히 올라가는지 
# topic 수를 다양도를 보면서 비교해보며 적절한 토픽의 수를 결정할수 있다. 